TASK:
I was tasked with creating a model which could predict the maximum temperature on a given day. To do this I started by attempting to implement a linear regression model to try to predict the maximum temperature on a given day. However this was relatively inaffective. I used this code, which calls a linear regression class.

IMPORTANT LINK: https://github.com/rowanwhelan/WeatherPerson

In [ ]:
import csv
from datetime import datetime, timedelta
import numpy as np
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from keras.models import model_from_json
from meteostat import Point, Daily
from keys.OpenWeather import key
import pandas as pd

In [ ]:
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

#/
def gradientDescent(Theta0, learnrate, function, partial, epsilon):
    theta = Theta0
    t = 0 
    while True:
        t = t + 1
        thetaTemp = theta
        theta = theta - learnrate(partial(theta))
        if math.abs(function(theta) - function(thetaTemp)) < epsilon:
            break
    return theta
#/

class Regression:
    def __init__(self, iterations: int = 100, learning_rate: float = 1, dimension: int = 1):
        self.iterations = iterations
        self.alpha = learning_rate

        self.intercept = 0
        self.weight_vector = [0] * dimension

    def fit(self, x, y):    
        #initialize parameters 
        intercept = 0
        weight_vector = [ random.random() for col in range(len(x[0])) ]


        #begin fitting data
        iteration_number = 0 
        while iteration_number < self.iterations:
            iteration_number += 1
            intercept_gradient, weight_vector_gradient = self.compute_gradient(x, y, intercept, weight_vector)
            intercept, weight_vector = self.update(intercept, intercept_gradient, weight_vector, weight_vector_gradient)
        return intercept, weight_vector
        

    def compute_gradient(self, x,y,intercept, weight_vector):
        intercept_gradient, weight_vector_gradient = 0, [0] * len(x[0])
        for i in range(len(x)):
            #predict with your model then see how accurate your prediction is
            y_i_hat = intercept + sum((x[i][j] * weight_vector[j]) for j in range(len(x[0])))
            partial_error = 2 * (y[i] - y_i_hat)

            for j in range(len(x[0])):
                #update your values based on your predictions 
                weight_vector_gradient[j] += partial_error * (x[i][j]) / len(x)
                intercept_gradient += partial_error/ len(x)
        return intercept_gradient, weight_vector_gradient
    
    def update(self, intercept, intercept_gradient, weight_vector, weight_vector_gradient):
        intercept += intercept_gradient * self.alpha
        for j in range(len(weight_vector)):
            weight_vector[j] += weight_vector_gradient[j] * self.alpha
        return intercept, weight_vector


In [ ]:
from Regression import Regression


reg = Regression(1000000,.0001,1)

belvedere_castle_tempartures = [61,41,47,55,67,58,49,52,54]
midway_international_airport_temperatures = [53,45,46,61,73,72,44,48,49]
bergstrom_temperatures = [70,49,69,82,79,84,90,83,75]
miami_temperatures = [79,81,79,79,82,84,82,80,86]

z = [ [data] for data in range(1,10)]

belv_int, belv_values = reg.fit(z,belvedere_castle_tempartures)
belv_prediction = belv_int + belv_values[0]*6
print("the model predicts:", belv_prediction, "for belvedere")

reg2 = Regression(1000000,0.0001,1)
mid_int, mid_values = reg2.fit(z,midway_international_airport_temperatures)
mid_predicition = mid_int + mid_values[0]*6
print("the model predicts:", mid_predicition, "for midway")

reg3 = Regression(1000000,0.0001,1)
berg_int, berg_values = reg3.fit(z,bergstrom_temperatures)
berg_prediction = berg_int + berg_values[0]*6
print("the model predicts:", berg_prediction, "for bergstrom")

reg4 = Regression(1000000,0.0001,1)
miami_int, miama_values = reg4.fit(z,miami_temperatures)
miami_prediction = miami_int + miama_values[0]*6
print("the model predicts:", miami_prediction, "for miami")

However, the weather too complicated to be modeled by a simple linear regression model. This is reflected in the accuracy of this model.
Belvedere: 1/4 days correct        Average Loss: 4.25 degrees
Midway: 1/4 days correct           Average Loss: 11.75 degrees
Bergstrom: 0/4 days correct        Average Loss: 3.5 degrees
Miami: 1/4 days correct            Average Loss: 2.5 degrees
Clearly this would not be good enough to gamble on. So I decided to instead try to capture the changing hot and cold fronts with different meteorological data.

I decided to capture these changing phenomenon with the following features ( Previous Days Max Temperature, Precipitation, Wind Direction, Wind Speed, Air Pressure, Precipitation factoring in snow, Sunlight, Humidity, Average co2 in the air over a 24 hour span, and the change in co2 over 24 hours )

To do this I had to get this data from online sources. I used these methods.

In [ ]:
from datetime import datetime
import math
import time
import numpy as np
import requests,sys 
sys.path.append("../")
from CS_546.keys.NCEI import key

def write_data(data, path):
    if len(data) == 0:
        print("Error")
        return
    try:
        # Save the data to the CSV file
        np.savetxt(path, data, delimiter=",")
        print(f"Data has been successfully written to '{path}'.")
    except Exception as e:
        print("An error occurred while writing to the CSV file:", e)

def index_of_day(date_str):
    date_str = date_str[0:10]
    try:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        # Get the day of the year
        day_of_year = date_obj.timetuple().tm_yday
        year = int(date_str[0:4])
        day_of_year = day_of_year + ((-2000 + year)*365 + math.floor((-2000 + year) / 4))
        return day_of_year
    except ValueError:
        # Handle invalid date string
        print("Invalid date format. Please provide date in YYYY-MM-DD format.")

def tabulate_response(response,table):
    if response.status_code == 200:
        data = response.json()
        if data == {}:
            return table
        for observation in data['results']:
            date = observation['date']    
            value = observation['value']
            print(value, index_of_day(date)-1)
            table[index_of_day(date)-1][1] = value
    else:
        print(f'Error: {response.status_code} - {response.text}')
    return table

def collect_NCEI_data(station,data_set,path):
    retry_delay = 1
    table = np.array([[1,0]])
    for num in range (1,8765):
        table = np.vstack((table,[[num+1,0]]))
        
    for i in range(2000, 2023):
        time.sleep(retry_delay)
        start_date = str(i)+'-01-01'
        end_date = str(i)+'-12-31'
        api_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data'
        params = {
            'datasetid': data_set,
            'startdate': start_date,
            'enddate': end_date,
            'stationid': station,
            'format': 'json',
            'limit': '1000',
            'datatypeid': ['PRCP']
        }
        headers = {'token': key}
        response = requests.get(api_url, params=params, headers=headers)
        table = tabulate_response(response, table)
    write_data(table,path)
    return table


In [ ]:
import requests,sys 
sys.path.append("../")

def collect_NWS_data():
    #Belvedere
    stationid = 'KW035'
    start = '1980-01-01T00:00:00+19:00'
    end = '2023-12-21T24:00:00+19:00'
    api_url = f'https://api.weather.gov/stations/{start, end, stationid}/observations'
    response = requests.get(api_url, headers= {'User-Agent': 'WeatherPredictorApp/1.0 contact rwhelan340@gmail.com', 'Accept': 'application/json'})

    
    # Check if request was successful
    if response.status_code == 200:
    # Parse the JSON response
        data = response.json()
    # Print the entire response for inspection
        print(data)
    else:
        print('Error:', response.status_code)
        print('Response Content:', response.text)  # Print response content for inspection



In [ ]:
from datetime import datetime
import math
import sys
import time
import numpy as np
import pyowm
from pyowm.utils.config import get_config_from
sys.path.append("../")
from CS_546.keys.OpenWeather import key

def get_co_in_air(latitude, longitude, start_date, end_date):
    config_dict = pyowm.utils.config.get_default_config()
    owm = pyowm.OWM(key,config_dict)
    mgr = owm.airpollution_manager()
    co2_vals = np.array([])
    retry_count = 5
    retry_delay = 2 # seconds

    for _ in range(retry_count):
        try:
            list_of_historical_values = mgr.air_quality_history_at_coords(latitude, longitude, start=start_date, end=end_date)
            # Each item in the list_of_historical_values is an Air  Status object 
            for air_status in list_of_historical_values:
                co2_vals = np.append(co2_vals,air_status.co)
            break

        except pyowm.commons.exceptions.NotFoundError:
            print("Historical weather data not found for the specified location and date.")
            break
        except Exception as e:
            print("An error occurred:", e)
            print("Retrying...")
            time.sleep(retry_delay)
    else:
        print("Failed to retrieve historical weather data after multiple retries.")
    
    return co2_vals

def write_data(data, path):
    if len(data) == 0:
        print("Error")
        return
    try:
        # Save the data to the CSV file
        np.savetxt(path, data, delimiter=",")
        print(f"Data has been successfully written to '{path}'.")
    except Exception as e:
        print("An error occurred while writing to the CSV file:", e)
        
def collect_co2_data(latitude, longitude, path):
    complete_co2_table = np.array([])
    start = datetime(2000,1,1)
    end = datetime(2023,12,31)
    complete_co2_table = np.append(complete_co2_table, get_co_in_air(latitude, longitude,start,end_date = end))
    print(len(complete_co2_table))
    #write_data(complete_co2_table, path)
    return


In [ ]:
from datetime import datetime
from meteostat import Point, Daily
import numpy as np

def collect_air_stats(latitude, longitude, path):
    location = Point(latitude,longitude)
    data = Daily(location, start = datetime(2000,1,1,0,0), end= datetime(2023,12,31,0,0))
    data = data.fetch()
    data = data[['tmax', 'prcp', 'wdir', 'wspd', 'pres']]
    data = data.fillna(-1)
            
    np.savetxt(path, data, delimiter=',')
    print('array successfully saved')
    


In [ ]:
import math
from matplotlib import pyplot as plt
import numpy as np, datetime, ephem

#function to convert from year day to the ephem internal values
def date_to_ephem_date(year,day):
    return ephem.Date(36525.0 + ((-2000 + year) *365) + day + math.floor((-2000 + year) / 4) -0.5)

def collect_sunlight_hours(latitude, longitude, print_val, path):
    observer = ephem.Observer()
    observer.lat, observer.lon = latitude, longitude
    all_dates = np.array([])
    all_daylight_hours = np.array([])
    all_dates, all_daylight_hours = create_sunlight_table(observer, all_dates, all_daylight_hours)
    if print_val:
        plot_sunlight_table(all_dates, all_daylight_hours)
    zipped_array = np.vstack((all_dates, all_daylight_hours)).ravel(order='F')
    np.savetxt(path, all_daylight_hours, delimiter=',')
    print('array successfully saved')

def create_sunlight_table(observer, all_dates, all_daylight_hours):
    for year in range(2000,2024):
        dates = np.array([])
        daylight_hours = np.array([])
        days = 366
        if(year%4 == 0):
            days = days+1
        for day in range(1,days):
            date = f'{year}-{day:03d}'
            observer.date = datetime.datetime.strptime(date, '%Y-%j')
            sunrise = observer.next_rising(ephem.Sun())
            sunset = observer.next_setting(ephem.Sun())
            #The sunset on the day happens in the morning before the sunrise, so we must update the day to compensate for this
            if sunset - sunrise < 0:
                #There is a fringe case where if the sunset happens very close to the changing of days, so this if statement catches that case and handles it by removing 24 hours from that calculation
                if sunset - date_to_ephem_date(year, day) < 0.007:
                    sunset = sunset + 1
                #Otherwise the algorithm continues as normal finding the next sunset
                else:
                    date = f'{year}-{day+1:03d}'
                    if day == days-1:
                        date = f'{year+1}-{1:03d}'
                    observer.date = datetime.datetime.strptime(date, '%Y-%j')
                    sunset = observer.next_setting(ephem.Sun())
            #Now that edge cases have been accounted for the algorithm computes the amount of sunlight and appends it to the return array   
            daylight_duration = sunset - sunrise
            daylight_hours = np.append(daylight_hours, daylight_duration * 24)
            dates = np.append(dates,int(date_to_ephem_date(year,day)))  
        all_dates = np.append(all_dates,dates)
        all_daylight_hours = np.append(all_daylight_hours,daylight_hours)
    return all_dates, all_daylight_hours

def plot_sunlight_table(all_dates, all_daylight_hours):
    plt.figure(figsize=(15, 8))
    plt.plot(all_dates, all_daylight_hours, label='Daylight')
    plt.title('Daylight Hours')
    plt.xlabel('Date')
    plt.ylabel('Daylight Hours')
    plt.show()
    


It is important to mention that Austin, Texas, doesn't record CO2 data, so every method will have two versions to accommodate the different data. I will analyze whether this affects accuracy later.

The overall structure I used to train my models follows the pattern of compiling the table, formatting it into training and testing data, and then training and testing the model.

In [ ]:
def complete_NN(name, prev):
    print(f'Training a model for {name}')
    data = compile_tables(name, prev)
    testing_labels, testing_data, data_labels, data = format_data(data)
    model = NN(data.astype(float), data_labels.astype(float))
    test_model(model,testing_data.astype(float),testing_labels.astype(float))
    return model

In [ ]:
def complete_Berg_NN(name, prev):
    print(f'Training a model for {name}')
    data = compile_tables(name, prev)
    testing_labels, testing_data, data_labels, data = format_data(data)
    model = Berg_NN(data.astype(float), data_labels.astype(float))
    test_model(model,testing_data.astype(float),testing_labels.astype(float))
    return model

Now that the general layout of the solution is clear I will explain each of the four methods. The first of these methods opens the csv files the data is saved as an compiles the data into one table.

In [ ]:
def compile_tables(name,prev_temp):
    filename = f'C:/Users/rwhel/OneDrive/Desktop/Notes/College/CS_546/data/{name}_Air_Pressure.csv'
    meteo_array = read_data(filename)
    filename = f'C:/Users/rwhel/OneDrive/Desktop/Notes/College/CS_546/data/{name}_NCEI.csv'
    ncei_array = read_data(filename)
    filename = f'C:/Users/rwhel/OneDrive/Desktop/Notes/College/CS_546/data/{name}_Sunlight.csv'
    sun_array = np.genfromtxt(filename, delimiter=',')
    # sun array data is taken farther back, but I have decided to only use the last 23 years of data to match the other sources
    original_shape = (2, len(sun_array) // 2)
    unzipped_array = sun_array.reshape(original_shape, order='F')
    sun_array = unzipped_array[1]
    padded_sun_array = np.pad(sun_array, (8765-8401, 0), 'constant', constant_values=0)
    filename = f'C:/Users/rwhel/OneDrive/Desktop/Notes/College/CS_546/data/{name}_humidity.csv'
    #humidity data only goes back to 2009 so you need to add 
    humid_array = read_data(filename)
    humid_array = humid_array[:,1]
    padded_humid_array = np.pad(humid_array, ((3290), (0)), mode='constant', constant_values=0)
    
    temps = meteo_array[:,0]
    hist_temp = create_previous_temparatures(temps, prev_temp)

    
    if name != 'Bergstrom':
        filename = f'C:/Users/rwhel/OneDrive/Desktop/Notes/College/CS_546/data/{name}_Air_c02.csv'
        co2_table = read_data(filename)
        co2_table = co2_table[-26295:]
        co2_table = co2_reshape(co2_table)
        combined_array = np.column_stack((meteo_array[:, 0], celsius_to_fahrenheit(hist_temp[:,0]), celsius_to_fahrenheit(hist_temp[:,1]), celsius_to_fahrenheit(hist_temp[:,2]), meteo_array[:, 1], meteo_array[:, 2], meteo_array[:, 3], meteo_array[:, 4], ncei_array[:, 1], padded_sun_array, padded_humid_array, co2_table[:, 0], co2_table[:, 1]))
        return combined_array
    else:
        combined_array = np.column_stack((meteo_array[:, 0], celsius_to_fahrenheit(hist_temp[:,0]), celsius_to_fahrenheit(hist_temp[:,1]), celsius_to_fahrenheit(hist_temp[:,2]), meteo_array[:, 1], meteo_array[:, 2], meteo_array[:, 3], meteo_array[:, 4], ncei_array[:, 1], padded_sun_array, padded_humid_array))
        return combined_array
# pass in a n x 1 np array and three manually inputted temperatures get back a n x 3 np array
def create_previous_temparatures(data, temps):
    second = np.array([temps[1], temps[2], data[0]])
    third = np.array([temps[2], data[0],data[1]])
    final = np.row_stack([temps,second,third])
    for i in range(3,len(data)):
        slice = np.array([data[i-3], data[i-2], data[i-1]])
        final = np.row_stack([final, slice])
    return final
# CO2 data was taken every 8 hours, so I decided to have the algorithm learn based on the slope of the day and the average co2        
def co2_reshape(table):
    # Reshape the original table to have three columns
    reshaped_table = table.reshape(-1, 3)

    # Calculate the average of every three points
    averages = np.mean(reshaped_table, axis=1)

    # Calculate the slope between each pair of consecutive points
    slopes = np.diff(reshaped_table, axis=1)[:, 1] / 2  # Calculate the average of the differences as the slope

    # Stack the averages and slopes to create the final 8765 x 2 table
    final_table = np.column_stack((averages, slopes))
    return final_table

Now that the method to get the tables is defined, the data has to be divided into testing and training sets. This is done by arbitrarily selected the last 100 days to be testing data.

In [ ]:
def format_data(data):
    labels = (data[:,0].astype(float) * 1.8) + 32
    testing_labels = labels[-100:]
    labels = labels[0:len(labels)-100]
    data = data[:,1::]
    testing_data = data[-100:]
    data = data[0:len(data)-100]
    return testing_labels, testing_data, labels, data

Now the data is fully formated and we can create our model.

In [ ]:
class ClimateNN(nn.Module):
        def __init__(self):
            super(ClimateNN, self).__init__()
            self.fc1 = nn.Linear(12, 128)  # Input layer
            self.fc2 = nn.Linear(128,64)
            self.fc3 = nn.Linear(64, 32)   # Hidden layer
            self.fc4 = nn.Linear(32, 1)    # Output layer

        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = torch.relu(self.fc3(x))
            x = self.fc4(x)
            return x
        
        def predict(self, input_data):
            input_tensor = torch.tensor(input_data, dtype=torch.float32)
            with torch.no_grad():
                self.eval()  # Set model to evaluation mode
                output_tensor = self(input_tensor)
                # If you have a single output neuron (e.g., regression), you might do:
                predicted_output = output_tensor.item()
                return predicted_output

def NN(data,labels):
    # Convert numpy arrays to PyTorch tensors
    X = torch.tensor(data, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)


    # Initialize the model, loss function, and optimizer
    model = ClimateNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Training the model
    epochs = 400
    batch_size = 32
    for epoch in range(epochs):
        for i in range(0, len(X), batch_size):
            inputs = X[i:i+batch_size]
            batch_labels = y[i:i+batch_size].view(-1,1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('Finished Training')
    return model

In [ ]:
class BergClimateNN(nn.Module):
        def __init__(self):
            super(BergClimateNN, self).__init__()
            self.fc1 = nn.Linear(10, 128)  # Input layer
            self.fc2 = nn.Linear(128,64)
            self.fc3 = nn.Linear(64, 32)   # Hidden layer
            self.fc4 = nn.Linear(32, 1)    # Output layer

        def forward(self, x):
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = torch.relu(self.fc3(x))
            x = self.fc4(x)
            return x
        
        def predict(self, input_data):
            input_tensor = torch.tensor(input_data, dtype=torch.float32)
            with torch.no_grad():
                self.eval()  # Set model to evaluation mode
                output_tensor = self(input_tensor)
                # If you have a single output neuron (e.g., regression), you might do:
                predicted_output = output_tensor.item()
                return predicted_output
def Berg_NN(data,labels):
    # Convert numpy arrays to PyTorch tensors
    X = torch.tensor(data, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)


    # Initialize the model, loss function, and optimizer
    model = BergClimateNN()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Training the model
    epochs = 400
    batch_size = 32
    for epoch in range(epochs):
        for i in range(0, len(X), batch_size):
            inputs = X[i:i+batch_size]
            batch_labels = y[i:i+batch_size].view(-1,1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('Finished Training')
    return model

The NN and BergNN methods are called to instantiate ClimateNN and BergClimateNN objects, train, and return them. This part of the process required a couple of design decisions. Each model has four layers, which start by increasing the dimensions of the data before shrinking them. This was just what I found to work best with the data I gathered. Secondly, the models are trained for 400 epochs; this choice was made because I started dramatically overfitting my data at 1000 epochs. Finally, here is the code to test the models:

In [ ]:
def test_model(model, test_data, test_labels, threshold=0.4):
    # Convert test data to PyTorch tensor
    X_test = torch.tensor(test_data, dtype=torch.float32)
    y_test = torch.tensor(test_labels, dtype=torch.float32).view(-1,1)

    # Set the model to evaluation mode
    model.eval()

    # Pass test data through the model
    with torch.no_grad():
        outputs = model(X_test)

    # Calculate loss (optional, depending on your requirements)
    criterion = nn.MSELoss()
    test_loss = criterion(outputs, y_test).item()
    print(f'Test Loss: {test_loss:.4f}')

    # Evaluate performance
    num_correct = 0
    for pred, true_label in zip(outputs, y_test):
        print('predicted:', pred, 'label:', true_label)
        if torch.abs(pred - true_label) <= threshold:
            num_correct += 1

    accuracy = num_correct / len(test_labels) * 100
    print(f'Percentage of Correctly Labeled Test Data: {accuracy:.2f}%')

    return accuracy

Now that the models have been fully created, there is an obvious question remaining: do the improve on the linear regression models?

CONCLUSIONS:
    Belvedere| Training Loss: 3.76 degrees Test Loss: 6.62 degrees
    Midway| Training Loss: 4.49 degrees Test Loss: 7.92 degrees
    Bergstrom| Training Loss: 2.19 degrees Test Loss: 3.34 degrees
    Miami| Training Loss: 3.76 degrees Test Loss: 6.37 degrees

These conclusions are not as promising as expected. It is essential to highlight that outliers skew the test loss. However, the models are still not much more accurate than a linear regression model. However, this might not be true when applied in real world settings.

The final part of the task was to code automated predictions. The strategy I used to do this was to first forecast all of the data I needed to make a prediction for that day, then have the model predict the data and bet on the day.

In [ ]:
def get_forecast(latitude, longitude, name):
    today = datetime.now()
    # Create a Point object with the coordinates
    location = Point(latitude, longitude)

    # Fetch forecast data from OpenWeatherMap API
    api_key = key
    if name == 'Belvedere':
        response = requests.get(f'http://pro.openweathermap.org/data/2.5/weather?q=New York City, New York&APPID={key}')
    elif name == 'Midway':
        response = requests.get(f'http://pro.openweathermap.org/data/2.5/weather?q=Chicago, Illinois&APPID={key}')
    elif name == 'Bergstrom':
        response = requests.get(f'http://pro.openweathermap.org/data/2.5/weather?q=Austin, Texas&APPID={key}')
    else:
        response = requests.get(f'http://pro.openweathermap.org/data/2.5/weather?q=Miami, FLorida&APPID={key}')
    openweather_data = response.json()
    
    meteo_data = Daily(location, today)
    meteo_data = meteo_data.normalize()
    meteo_data = meteo_data.fetch()
    
    #Temperature array
    today = pd.Timestamp.now().normalize()
    yesterday = today - pd.Timedelta(days=1)
    max_temp_yesterday = Daily(location, yesterday).normalize().fetch()[meteo_data.index == yesterday.strftime('%Y-%m-%d')]['tmax'].values[0]
    day_before_yesterday = yesterday - pd.Timedelta(days=1)
    max_temp_day_before = Daily(location, day_before_yesterday).normalize().fetch()[meteo_data.index == day_before_yesterday.strftime('%Y-%m-%d')]['tmax'].values[0]
    day_before_day_before_yesterday = day_before_yesterday - pd.Timedelta(days=1)
    max_temp_day_before_day_before = Daily(location, day_before_day_before_yesterday).normalize().fetch()[meteo_data.index == day_before_day_before_yesterday.strftime('%Y-%m-%d')]['tmax'].values[0]
    
    prev_temp = [celsius_to_fahrenheit(max_temp_yesterday), celsius_to_fahrenheit(max_temp_day_before), celsius_to_fahrenheit(max_temp_day_before_day_before)]
        
    #PRCP
    precipitation_rain = meteo_data[meteo_data.index == today.strftime('%Y-%m-%d')]['prcp'].fillna(0).values[0]
    precipitation_snow = meteo_data[meteo_data.index == today.strftime('%Y-%m-%d')]['snow'].fillna(0).values[0]
    precipitation = precipitation_snow + precipitation_rain
    
    #Humidity
    humidity = openweather_data['main']['humidity']
    
    #sunlight
    sunrise_timestamp = openweather_data['sys']['sunrise']
    sunrise_datetime = datetime.fromtimestamp(sunrise_timestamp)
    sunset_timestamp = openweather_data['sys']['sunset']
    sunset_datetime = datetime.fromtimestamp(sunset_timestamp)
    sunlight_duration_hours = datetime_to_hours_float(sunset_datetime - sunrise_datetime)
    
    #Wdir
    wind_direction_degrees = meteo_data[meteo_data.index == today.strftime('%Y-%m-%d')]['wdir'].fillna(0).values[0]
    
    #WPGT
    wind_speed = meteo_data[meteo_data.index == today.strftime('%Y-%m-%d')]['wspd'].fillna(0).values[0]
    
    #Pres
    pressure = meteo_data[meteo_data.index == today.strftime('%Y-%m-%d')]['pres'].fillna(0).values[0]
    
    #co2 in air 
    # the datetime objects here are midnight yesterday and midnight today
    co2_average = None
    co2_change = None
    if name != 'Bergstrom':
        air_quality_history = get_co_in_air(latitude, longitude, datetime.combine(yesterday, datetime.min.time()), datetime.combine(today, datetime.min.time()))
        co2_levels = [entry for entry in air_quality_history]
        co2_values = [co2_levels[0], co2_levels[8], co2_levels[16]]
        co2_average = sum(co2_values)/3
        co2_change = co2_values[2] - co2_values[0]
    
    # Print forecast data
    #print("Forecast for today:")
    #print(f"Previous Temperatures: {prev_temp}")
    #print(f"Precipitation: {precipitation} mm")
    #print(f"Wind Direction: {wind_direction_degrees}")
    #print(f"Wind Peak Gust: {wind_speed} m/s")
    #print(f"Pressure: {pressure}")
    #print(f"Humidity: {humidity}")
    #print(f"Sunlight Duration: {sunlight_duration_hours} hours")
    #print(f"Average CO2: {co2_average} ppm")
    #print(f"Change in CO2: {co2_change}")
    #Previous Temps, prcp, wdir, wpgt, pres, prcp, sunlight, humidity, co2 average, co2 slope
    if name != 'Bergstrom':
        return np.array([prev_temp[0], prev_temp[1], prev_temp[2], precipitation, wind_direction_degrees, wind_speed, pressure, precipitation, sunlight_duration_hours, humidity, co2_average, co2_change ])
    else:
        return np.array([prev_temp[0], prev_temp[1], prev_temp[2], precipitation, wind_direction_degrees, wind_speed, pressure, precipitation, sunlight_duration_hours, humidity])

Next we use our trained models to predict the temperature.

In [ ]:
def generate_prediction( latitude, longitude, name, model):
    forecast = get_forecast(latitude, longitude,name)
    return model.predict(forecast)

Finally we have the computer place automatic bets.

In [ ]:
def bet_on_prediction(name, prediction):
    config = kalshi_python.Configuration()
    config.host = 'https://demo-api.kalshi.co/trade-api/v2'
    kalshi_api = kalshi_python.ApiInstance(
        email= email,
        password= password,
        configuration=config,
    )
    market_ticker = ''
    side = ''
    if name == 'Midway':
        market_ticker = 'HIGHCHI-24APR02-T38'
        if prediction < 38:
            side = 'yes'
        else:
            side = 'no'
    elif name == 'Bergstrom':
        market_ticker = 'HIGHAUS-24APR02-T75'
        if prediction > 75:
            side = 'no'
        else:
            side = 'yes'
    elif name == 'Belvedere':
        market_ticker = 'HIGHNY-24APR02-B47.5'
        if prediction < 48 and prediction > 47:
            side = 'yes'
        else:
            side = 'no'
    else:
        market_ticker = 'HIGHMIA-24APR02-T79'
        if prediction > 79:
            side = 'no'
        else:
            side = 'yes'
    exchange_status = kalshi_api.get_exchange_status()
    print('Exchange status response: ')
    print(exchange_status)
    if exchange_status.trading_active:
        # Submit an order for 10 yes contracts at 50cents on 'FED-23DEC-T3.00'.
        order_uuid = str(uuid.uuid4())
        order_response = kalshi_api.create_order(CreateOrderRequest(
            ticker=market_ticker,
            action='buy',
            type='limit',
            yes_price=50,
            count=500,
            client_order_id=order_uuid,
            side=side,
        ))
        print('\nOrder submitted: ')
        pprint(order_response)
    else:
        print('\nThe exchange is not trading active, no orders will be sent right now.')

EFFECTIVENESS:
I started the model with 5000 dollars and ran it for five days. Over that period the change in networth was:
    1: -132
    2: 11
    3: 269
    4: -240 
    5: 1
For a total change of -98. This was not a superb showing for the model, but with my limited testing I did discover that a couple of the features (precipitation and wind direction) were causing wild skews in the data. This implies that fundemental idea behind the model was sound, but the execution of the idea was flawed. 